## mimlsvm

mi = mulit-instance
ml = multi-label
svm = svm

As described in Shen et al 2009: http://ieeexplore.ieee.org/document/5346261/

> Should we use SVM-struct instead? https://github.com/pystruct/pystruct


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
import nltk
import os
import re
import pickle
import sklearn
import sys
import string

from hausdorff import hausdorff

from nltk import TextTilingTokenizer
from scipy.spatial.distance  import directed_hausdorff, pdist

from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import cross_val_score, GridSearchCV,ParameterGrid, train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler,MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer
from sklearn.svm import SVC

from tqdm import *

%matplotlib inline
%load_ext autoreload
%autoreload 1

In [2]:
src_dir = os.path.join(os.getcwd(), os.pardir, '../../')
sys.path.append(src_dir)

In [3]:
%aimport src.data.movielens_20m_imdb
%aimport src.helpers.labels,src.helpers.neighbours, src.helpers.segments
%aimport src.utils.dataframes, src.utils.clusters, src.utils.metrics

In [4]:
from src.data.movielens_20m_imdb import load_df_or_get_from_cache
from src.helpers.labels import truncate_labels
from src.helpers.neighbours import get_predicted_labels_from_neighbours
from src.helpers.segments import make_distance_matrix_for_segments,vectorize_segments

from src.utils.dataframes import sample_rows
from src.utils.metrics import ranking
from src.utils.clusters import k_medoids

In [39]:
MODELS_ROOT = os.path.abspath("../../../models/ranking/movielens-ovr-linear-svc-calibrated/")
INTERIM_DATA_ROOT = os.path.abspath("../../../data/interim/movielens-ml20m-imdb/")
PATH_TO_PROCESSED_FILE = os.path.abspath('../../../data/processed/movielens-20m-imdb-tags-and-synopses-2017-12-20.csv')

# CONFIGS

SEED= 42
MAX_NB_WORDS = 500
NB_DOCS=500

# preprocessing

PREPROC=None
STOP_WORDS=[None,'english']
VECTORIZER_NORM = ['l1','l2',None]

W=20 # Pseudosentence size (in words) - not specified in the paper, taken from TextTiling default values
K=10 # Size (in sentences) of the block used in the block comparison method - not specified in the paper, taken from TextTiling default values

# calculating medoids
MEDOID_NORMALIZATION = ['standard','minmax',None] # not specified in the paper
SAMPLE_TO_NB_MEDOIDS_RATIO = [0.2,0.3,0.4] # not specified in the paper, but taken from MIMLSVM canonical implementation

# classification
SVM_KERNEL=['poly'] # not specified in the paper, but taken from MIMLSVM canonical implementation
SVM_GAMMA=[0.1,0.2,0.3,0.5,1.0,'auto'] # not specified in the paper, but taken from MIMLSVM canonical implementation
SVM_C= [0.001,0.01,0.1,1]# not specified in the paper, but taken from MIMLSVM canonical implementation
SVM_DEGREE=[3,4,5]

In [41]:
MIN_TAG_DF = 10

In [42]:
docs_df = load_df_or_get_from_cache(PATH_TO_PROCESSED_FILE,INTERIM_DATA_ROOT)

In [43]:
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [44]:
# remove this for production
docs_df = sample_rows(docs_df,NB_DOCS)

In [45]:
docs_df['sentences'] = docs_df['synopsis'].map(lambda row: sentence_tokenizer.tokenize(row))

In [46]:
%%time
docs_df['num_sentences'] = docs_df['sentences'].map( lambda sents: len(sents))

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.29 ms


In [47]:
docs_df.head()

,movie_id,title,synopsis,tags,num_tags,sentences,num_sentences
0,91831,"Devil Inside, The (2012)",An American girl sets out to make a documentar...,"religion,christianity,catholicism,psychiatry,e...",5,[An American girl sets out to make a documenta...,8
1,79430,8: The Mormon Proposition (2010),As California's anti-gay marriage proposition ...,"religion,queer,documentary,politics",4,[As California's anti-gay marriage proposition...,34
2,3083,All About My Mother (Todo sobre mi madre) (1999),"Manuela is a single mother of Madrid, who brin...","bibliothek,spanish,homosexuality,erlend's-dvds...",13,"[Manuela is a single mother of Madrid, who bri...",10
3,6725,Sgt. Pepper's Lonely Hearts Club Band (1978),The original Sgt. Pepper's Lonely Hearts Club ...,"steve-martin,70mm,anamorphic-blow-up",3,"[The original Sgt., Pepper's Lonely Hearts Clu...",82
4,8533,"Notebook, The (2004)","In a modern-day nursing home, an elderly man n...","romantic,rachel-mcadams,first-love,ryan-goslin...",32,"[In a modern-day nursing home, an elderly man ...",52


In [48]:
docs_df.iloc[0]['sentences']

['An American girl sets out to make a documentary to understand what happened to her mother, who murdered three clergy people but was not convicted due to insanity and was instead sent to a mental hospital in the Vatican City.',
 'She hires a camera man and they attend a seminar on exorcism in the Vatican City.',
 "She meets some priests who explain that her mother's condition may not be medical, but could well be demonic possession.",
 'She witnesses an exorcism with the priests, which is also caught on tape.',
 'The priests and the documentary team then decide to carry out an exorcism on her mother, during which they seek to discover whether her mother is actually possessed.',
 'It turns out that she was possessed by at least four demons, and during the exorcism there occurs some transference - the migration of the demon from one body to another.',
 'Three of them are possessed and soon after they end up dead.',
 'It ends abruptly, with a car crash due to the demonic possession of th

In [49]:
docs_df.describe()

,movie_id,num_tags,num_sentences
count,500.000000,500.000000,500.000000
mean,38550.468000,12.506000,56.396000
std,38476.692399,14.999064,76.354484
min,17.000000,1.000000,1.000000
25%,4098.500000,3.000000,13.000000
50%,26491.500000,7.000000,36.000000
75%,69899.000000,15.000000,71.000000
max,128604.000000,108.000000,1136.000000


In [50]:
tok = TextTilingTokenizer(w=W, k=K)

In [51]:
def extract_segments(candidates):
    
    try:
        # we must manually insert "\n\n" because this is how 
        # texttilingtokenizer requires candidate boundaries to be 
        # represented.
        segments = tok.tokenize("\n\n".join(candidates))
    except ValueError:
        # this happens when the candidate list is too small for the 
        # text tiling tokenizer to be able to find segments. so just return
        # the original sentences.
        segments= candidates
        
    # now remove the artificially added chars
    segments = [segment.replace("\n\n"," ").strip() for segment in segments]
    
    return segments

In [52]:
%%time
docs_df['segments'] = docs_df['sentences'].map(lambda candidates: extract_segments(candidates))

CPU times: user 2min 15s, sys: 80 ms, total: 2min 15s
Wall time: 2min 15s


In [53]:
docs_df['segments'][0][:3]

['An American girl sets out to make a documentary to understand what happened to her mother, who murdered three clergy people but was not convicted due to insanity and was instead sent to a mental hospital in the Vatican City.',
 "She hires a camera man and they attend a seminar on exorcism in the Vatican City. She meets some priests who explain that her mother's condition may not be medical, but could well be demonic possession. She witnesses an exorcism with the priests, which is also caught on tape. The priests and the documentary team then decide to carry out an exorcism on her mother, during which they seek to discover whether her mother is actually possessed.",
 'It turns out that she was possessed by at least four demons, and during the exorcism there occurs some transference - the migration of the demon from one body to another. Three of them are possessed and soon after they end up dead. It ends abruptly, with a car crash due to the demonic possession of the cameraman, who was

In [54]:
segments = docs_df['segments'].values
documents = docs_df['synopsis'].values
labels = docs_df["tags"].map(lambda tagstring: tagstring.split(","))

In [55]:
labels = truncate_labels(labels,MIN_TAG_DF)

In [56]:
def make_train_dataset(distance_matrix, medoid_indices):
    """
    Returns a matrix where element Aij contains the distance from sample i to medoid j.

    :param distance_matrix: MxM matrix with pairwise distances
    :param medoid_indices: array of length N containing the indices of the medoids for each cluster
    :return: distances to medoids (MxN matrix)
    """

    return distance_matrix[:,medoid_indices]

In [57]:
def make_test_dataset(source_vectorized_segments, medoid_vectorized_segments):
    """
    Calculates the distances from every source_document (reprsented by its segments) to every medoid
    document (also represented by its segments) using the hausdorff distance.
    
    Returns a matrix where element Aij contains the distance from sample i to medoid j.

    :param source_vectorized_segments: array of length M, where each element is a matrix with one row
        for every segment in a source document
    :param medoid_vectorized_segments: array of length N where each element is a matrix with one row
        for every segment in a medoid document
    :return: distances to medoids (MxN matrix)
    """
    
    num_test_samples = len(source_vectorized_segments)
    num_medoids = len(medoid_vectorized_segments)
    
    test_dataset = np.zeros((num_test_samples,num_medoids))    
    
    for i,source_segments in enumerate(source_vectorized_segments):
        for j,medoid_segments in enumerate(medoid_vectorized_segments):
            test_dataset[i][j] = hausdorff(source_segments.toarray(),medoid_segments.toarray())
            
    return np.array(test_dataset)

In [58]:
# segments, documents and labelsets are defined outside of the parameterGrid loop
# because they're the same for every configuration    
segments_train, segments_test, documents_train, documents_test, Y_train, Y_test = train_test_split(segments,
                                                                                               documents,
                                                                                               labels,
                                                                                               test_size=0.15)

print('total number of train documents: {}'.format(len(documents_train)))
print('total number of validation documents: {}'.format(len(documents_test)))

total number of train documents: 425
total number of validation documents: 75


In [59]:
mlb = MultiLabelBinarizer()


# the binarizer needs to be fit on all labels
mlb = MultiLabelBinarizer()
mlb.fit(labels)

Y_train = mlb.transform(Y_train)
Y_test = mlb.transform(Y_test)

print("total number of unique tags: {} ".format(len(mlb.classes_)))

total number of unique tags: 128 


In [60]:
parameters = [
    {
        'stop_words': STOP_WORDS,
        'medoid_normalization':  MEDOID_NORMALIZATION,
        'svm_kernel': ['poly'],
        'svm_gamma': SVM_GAMMA,
        'svm_c':SVM_C,
        'svm_degree' :SVM_DEGREE,
        'norm': VECTORIZER_NORM,
        'nb_medoids_ratio': SAMPLE_TO_NB_MEDOIDS_RATIO
    },
    {
        'stop_words': STOP_WORDS,
        'medoid_normalization':  MEDOID_NORMALIZATION,
        'svm_kernel': ['rbf'],
        'svm_gamma': SVM_GAMMA,
        'svm_c':SVM_C,
        'svm_degree' :[None],
        'norm': VECTORIZER_NORM,
        'nb_medoids_ratio': SAMPLE_TO_NB_MEDOIDS_RATIO
    }
]

In [61]:
print('total number of configurations to test: {}'.format(len(ParameterGrid(parameters))))

total number of configurations to test: 4320


In [66]:
best_score = float("-inf")
best_configuration = None

for (i,configuration) in enumerate(tqdm(ParameterGrid(parameters))):
    
    # TFIDF_VECTORIZER = COUNT_VECTORIZER + TFIDF_TRANSFORMER
    tfidf_vectorizer = TfidfVectorizer(max_features=MAX_NB_WORDS, 
                                       preprocessor=PREPROC, 
                                       stop_words=configuration['stop_words'],
                                       norm = configuration['norm'])
    
    # TRAINING SET
    tfidf_vectorizer.fit(documents)
    tfidf_segments_train = vectorize_segments(segments_train, tfidf_vectorizer)
    
    
    # THE FOLLOWING BLOCK TAKES SOME TIME, BUT IT WILL ONLY RUN ONCE
    norm = configuration['norm']
    stop_words = configuration['stop_words']
    
    path_to_cache = INTERIM_DATA_ROOT.rstrip('/') + "/mimlsvm/distance-matrix-train-sample-{}-{}-{}.p".format(
        NB_DOCS,
        norm,
        stop_words)
    
    if os.path.isfile(path_to_cache):
        dist_matrix_train = pickle.load(open(path_to_cache,"rb"))
    else:
        print('Fitting distance matrix for norm={} and stop_words={}'.format(norm,stop_words))
        
        dist_matrix_train = make_distance_matrix_for_segments(tfidf_segments_train)
        pickle.dump(dist_matrix_train, open(path_to_cache, "wb"))
    
    # nb_medoids depends upon the dataset length
    ratio = configuration['nb_medoids_ratio']
    nb_medoids = int(len(tfidf_segments_train) * ratio)
    
    medoids_indices_train = k_medoids(dist_matrix_train,nb_medoids)[0]

    X_train = make_train_dataset(dist_matrix_train,medoids_indices_train)
    
    # TEST SET
    
    tfidf_segments_test = vectorize_segments(segments_test, tfidf_vectorizer)
    # medoids trained on the training set
    fitted_medoids = tfidf_segments_train[medoids_indices_train]
    X_test = make_test_dataset(tfidf_segments_test,fitted_medoids)
    
    # RUNNING
    if configuration['medoid_normalization'] == 'standard':      
        scaler = StandardScaler()
        X_train_final = scaler.fit_transform(X_train)
        X_test_final = scaler.transform(X_test)
    elif configuration['medoid_normalization'] == 'minmax':
        scaler = MinMaxScaler()
        X_train_final = scaler.fit_transform(X_train)
        X_test_final = scaler.transform(X_test)
    else:
        X_train_final = X_train
        X_test_final = X_test
    
    clf = OneVsRestClassifier(
        SVC(kernel=configuration['svm_kernel'],
            gamma=configuration['svm_gamma'],
            C=configuration['svm_c'],
            degree=configuration['svm_degree'],
            probability=True,
           ),n_jobs=-1)    
        
    # y_train was defined outside the loop    
    clf.fit(X_train_final,Y_train)
    
    # train score
    Y_pred_train = clf.predict_proba(X_train_final)
    
    # validation score
    Y_pred_test = clf.predict_proba(X_test_final)  
    
    print("iter: {}, configuration: {}\n".format(i,configuration))
    

    ks = [1,2,3,4,5,6,7,8,9,10]

    for k in ks:
        print("train micro-F1 @{}: {}".format(k,ranking.micro_f1_at_k(Y_train,Y_pred_train,k=k)))
        print("validation micro-F1 @{}: {}".format(k,ranking.micro_f1_at_k(Y_test,Y_pred_test,k=k)))    


  0%|          | 0/4320 [00:00<?, ?it/s]




  0%|          | 1/4320 [00:03<4:14:00,  3.53s/it]

iter: 0, configuration: {'stop_words': None, 'svm_kernel': 'poly', 'svm_gamma': 0.1, 'nb_medoids_ratio': 0.2, 'medoid_normalization': 'standard', 'svm_c': 0.001, 'svm_degree': 3, 'norm': 'l1'}

train micro-F1 @1: 0.024550635686102586
validation micro-F1 @1: 0.03271028037383177
train micro-F1 @2: 0.039518253669552125
validation micro-F1 @2: 0.0486815415821501
train micro-F1 @3: 0.04939084622983207
validation micro-F1 @3: 0.06093189964157706
train micro-F1 @4: 0.056140350877192984
validation micro-F1 @4: 0.06549520766773163
train micro-F1 @5: 0.060415024954032046
validation micro-F1 @5: 0.06609195402298851
train micro-F1 @6: 0.06288708909004288
validation micro-F1 @6: 0.07217847769028872
train micro-F1 @7: 0.06424216027874564
validation micro-F1 @7: 0.07729468599033816
train micro-F1 @8: 0.06516944054541808
validation micro-F1 @8: 0.07683741648106904
train micro-F1 @9: 0.06516895655402896
validation micro-F1 @9: 0.07755946225439504
train micro-F1 @10: 0.06701298701298701
validation micro






  0%|          | 2/4320 [00:07<4:16:43,  3.57s/it]

iter: 1, configuration: {'stop_words': None, 'svm_kernel': 'poly', 'svm_gamma': 0.2, 'nb_medoids_ratio': 0.2, 'medoid_normalization': 'standard', 'svm_c': 0.001, 'svm_degree': 3, 'norm': 'l1'}

train micro-F1 @1: 0.03224381625441696
validation micro-F1 @1: 0.03271028037383177
train micro-F1 @2: 0.043445409897997736
validation micro-F1 @2: 0.0486815415821501
train micro-F1 @3: 0.05425074429374793
validation micro-F1 @3: 0.06093189964157706
train micro-F1 @4: 0.058927000879507474
validation micro-F1 @4: 0.06549520766773163
train micro-F1 @5: 0.06292785676671933
validation micro-F1 @5: 0.06609195402298851
train micro-F1 @6: 0.06567948411750657
validation micro-F1 @6: 0.0735873850197109
train micro-F1 @7: 0.06656481885639459
validation micro-F1 @7: 0.07729468599033816
train micro-F1 @8: 0.06795335745878568
validation micro-F1 @8: 0.07803790412486064
train micro-F1 @9: 0.06754745068850018
validation micro-F1 @9: 0.07533539731682147
train micro-F1 @10: 0.06867845993756504
validation micro-F1






  0%|          | 3/4320 [00:11<4:24:37,  3.68s/it]

...

  1%|          | 50/4320 [04:45<9:28:52,  7.99s/it]

iter: 49, configuration: {'stop_words': None, 'svm_kernel': 'poly', 'svm_gamma': 1.0, 'nb_medoids_ratio': 0.2, 'medoid_normalization': 'standard', 'svm_c': 1, 'svm_degree': 3, 'norm': 'l1'}

train micro-F1 @1: 0.01963350785340314
validation micro-F1 @1: 0.03271028037383177
train micro-F1 @2: 0.03484451105282878
validation micro-F1 @2: 0.0486815415821501
train micro-F1 @3: 0.042525351651946354
validation micro-F1 @3: 0.05714285714285714
train micro-F1 @4: 0.051222351571594875
validation micro-F1 @4: 0.06549520766773163
train micro-F1 @5: 0.05487326887901751
validation micro-F1 @5: 0.06609195402298851
train micro-F1 @6: 0.058349146110056926
validation micro-F1 @6: 0.07217847769028872
train micro-F1 @7: 0.06077707835901888
validation micro-F1 @7: 0.07729468599033816
train micro-F1 @8: 0.06133866133866134
validation micro-F1 @8: 0.07683741648106904
train micro-F1 @9: 0.06280103742126714
validation micro-F1 @9: 0.07755946225439504
train micro-F1 @10: 0.06369756602796478
validation micro-F1 

  1%|          | 51/4320 [04:50<8:17:24,  6.99s/it]

iter: 50, configuration: {'stop_words': None, 'svm_kernel': 'poly', 'svm_gamma': 0.1, 'nb_medoids_ratio': 0.2, 'medoid_normalization': 'standard', 'svm_c': 1, 'svm_degree': 4, 'norm': 'l1'}

train micro-F1 @1: 0.020969855832241154
validation micro-F1 @1: 0.03271028037383177
train micro-F1 @2: 0.03484451105282878
validation micro-F1 @2: 0.0486815415821501
train micro-F1 @3: 0.04491803278688525
validation micro-F1 @3: 0.06093189964157706
train micro-F1 @4: 0.05091649694501019
validation micro-F1 @4: 0.06549520766773163
train micro-F1 @5: 0.05459770114942529
validation micro-F1 @5: 0.07380607814761216
train micro-F1 @6: 0.058098174057386764
validation micro-F1 @6: 0.0735873850197109
train micro-F1 @7: 0.06077707835901888
validation micro-F1 @7: 0.07729468599033816
train micro-F1 @8: 0.0621875624875025
validation micro-F1 @8: 0.07683741648106904
train micro-F1 @9: 0.0633920296570899
validation micro-F1 @9: 0.07755946225439504
train micro-F1 @10: 0.06424870466321243
validation micro-F1 @10:

KeyboardInterrupt: 